### Project 3 EDA notebook 2 - further models

In [1]:
#imports

import pandas as pd 
import pandas_profiling
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score


In [2]:
#Read in the training data

train_data = pd.read_csv('../training_set_values', index_col=0)
train_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
train_labels = pd.read_csv('../training_set_labels', index_col=0)

In [4]:
train_df = pd.merge(train_labels, train_data, on=['id'])
train_df.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


#### It looks like a lot of these columns are similar and/or duplicates. Going to create a smaller dataframe without repeats.

In [5]:
small_train = train_df[['id', 'status_group', 'amount_tsh', 
                        'extraction_type_class', 'quality_group', 'quantity_group','population',
                       'source_class', 'waterpoint_type', 'basin','construction_year', 'region']]
small_train.shape

(59400, 12)

In [6]:
#many of these objects should be categories... let's change them

small_train['extraction_type_class'] = small_train['extraction_type_class'].astype('category')
small_train['quality_group'] = small_train['quality_group'].astype('category')
small_train['quantity_group'] = small_train['quantity_group'].astype('category')
small_train['source_class'] = small_train['source_class'].astype('category')
small_train['waterpoint_type'] = small_train['waterpoint_type'].astype('category')
small_train['basin'] = small_train['basin'].astype('category')
small_train['region'] = small_train['region'].astype('category')

<ipython-input-6-f09ed6c4d695>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_train['extraction_type_class'] = small_train['extraction_type_class'].astype('category')
<ipython-input-6-f09ed6c4d695>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_train['quality_group'] = small_train['quality_group'].astype('category')
<ipython-input-6-f09ed6c4d695>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [7]:
#how does that look?
small_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     59400 non-null  int64   
 1   status_group           59400 non-null  object  
 2   amount_tsh             59400 non-null  float64 
 3   extraction_type_class  59400 non-null  category
 4   quality_group          59400 non-null  category
 5   quantity_group         59400 non-null  category
 6   population             59400 non-null  int64   
 7   source_class           59400 non-null  category
 8   waterpoint_type        59400 non-null  category
 9   basin                  59400 non-null  category
 10  construction_year      59400 non-null  int64   
 11  region                 59400 non-null  category
dtypes: category(7), float64(1), int64(3), object(1)
memory usage: 3.1+ MB


In [9]:
#Some one hot encoding is in order

ohe = OneHotEncoder()
dummies = ohe.fit_transform(small_train[['extraction_type_class', 'quality_group', 'quantity_group',
                                         'source_class', 'basin', 'region']])

dummies_df = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names(),
                         index=small_train.index)
d_train_df = pd.concat([small_train[['status_group' ,'amount_tsh']], dummies_df], axis=1)
d_train_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   status_group                59400 non-null  object 
 1   amount_tsh                  59400 non-null  float64
 2   x0_gravity                  59400 non-null  float64
 3   x0_handpump                 59400 non-null  float64
 4   x0_motorpump                59400 non-null  float64
 5   x0_other                    59400 non-null  float64
 6   x0_rope pump                59400 non-null  float64
 7   x0_submersible              59400 non-null  float64
 8   x0_wind-powered             59400 non-null  float64
 9   x1_colored                  59400 non-null  float64
 10  x1_fluoride                 59400 non-null  float64
 11  x1_good                     59400 non-null  float64
 12  x1_milky                    59400 non-null  float64
 13  x1_salty                    594

In [10]:
#What does the data look like now?
d_train_df.shape

(59400, 53)

In [11]:
# Let's try a training model with all the features. Status Group is the target
y = d_train_df['status_group']
X = d_train_df.drop('status_group', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=47)

In [12]:
#Quick look at the y value counts.

y.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [13]:
#let's try a regression with all the features
logreg = LogisticRegression(solver= 'liblinear', random_state=42)
logreg.fit(X_train, y_train)
y_train_pred = logreg.predict(X_train)

In [14]:
logreg.predict(X_train)

y_pred = (logreg.predict(X_train))
recall_score(y_pred, y_train, average='weighted')

0.6958922558922559

In [15]:
cross_val_score(logreg, X_train, y_train, cv=5)

array([0.70056117, 0.69180696, 0.69775533, 0.69360269, 0.69113356])

In [16]:
print(cross_val_score(logreg, X_train, y_train, cv=5).mean())

0.6949719416386083


#### So 0.68-ish. Better than before. how does the 'average' parameter influence this?

### I want to try a plain ol Decision Tree

In [17]:
# Create the classifier, fit it on the training data and make predictions on the test set
clf = DecisionTreeClassifier(random_state=42, max_depth=63, max_features=10)

clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=63, max_features=10, random_state=42)

In [18]:
y_pred = (clf.predict(X_train))
recall_score(y_pred, y_train, average='weighted')

0.7665993265993266

##### We'll have to look closer at this in the morning and decide how to prune...